*<small>Last updated: 2026-02-25 01:35:27 UTC | Student Version (No Solutions)</small>*

**Student Version** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Instructor Version**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/gist/mtgca/7d52f0b7b63c6317f0151fe1505d85c7/char_rnn_classification_lightning.ipynb) &nbsp;&nbsp;&nbsp; [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mtgca/DL-Labs/blob/main/02%20Sequential%20Models/char_rnn_classification_lightning.ipynb)

# 📔 **Resolución**
Una vez que la nota sea entregada, encontrarás la resolución en este link de notion:

https://www.notion.so/R-Sequential-Models-30005737435780668286e83f53ce4780?source=copy_link

Recuerda que existen varias formar de solucionar las actividades propuestas en los laboratorios. Toma la resolución entregada como referencia. En caso de que el link de resolución no esté público, solicita acceso en el mismo link de notion.




# Character-Level RNN Classification with PyTorch Lightning

This notebook reimplements the character-level RNN name classification task using PyTorch Lightning, following best practices including:
- Proper train/val/test split
- LightningModule and LightningDataModule patterns
- Training metrics visualization
- Reproducibility with random seeds

Notebook based on https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

In [ ]:
import os
print(os.getcwd())

In [ ]:
# Instalar PyTorch Lightning si es necesario
!pip install lightning

# Downloading data

In [ ]:
![ ! -f data.zip ] && wget https://download.pytorch.org/tutorial/data.zip

In [ ]:
import os
import zipfile

# Check if data folder exists
print(os.getcwd())
if os.path.exists('./data'):
    print('Data folder already exists. Using existing data.')
    print('(If you need to re-extract, delete the data folder first)')
else:
    print('Unzipping data for the first time...')
    with zipfile.ZipFile('data.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    os.remove('data.zip')
    print('Done!')

In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchmetrics import Accuracy, F1Score

import string
import unicodedata
import glob
import os
from io import open
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Enable Tensor Cores on CUDA devices for better performance
torch.set_float32_matmul_precision('medium')

# Set random seeds for reproducibility
pl.seed_everything(2024, workers=True)

# Performance Optimization Through Environment Detection
The fundamental difference between VSCode remote containers and Google Colab lies in how they communicate with the user interface. In VSCode remote environments, your code executes inside a Docker container or remote server, and all output—including progress bar updates, print statements, and data transfers—must traverse the network to reach your local VSCode client. When PyTorch Lightning's progress bar updates hundreds of times per epoch (once per batch), these frequent UI updates create massive network overhead, with each update requiring a round-trip between the container and your client. Similarly, spawning multiple data loader worker processes in a remote container creates severe inter-process communication (IPC) bottlenecks, as each worker must serialize and transfer data through multiple layers. In contrast, Google Colab runs in a browser with a local connection to its backend servers, making these operations far less costly.

This environment detection code automatically identifies whether you're in a constrained remote environment and adjusts critical settings accordingly. For VSCode remote, it sets num_workers=0 to eliminate multi-process overhead (letting the GPU handle parallelism instead), disables progress_bar to eliminate hundreds of network round-trips per epoch, and reduces logging frequency by 10x. These changes transform training from 10-100x slower than native to near-native GPU speed—typically resulting in 5-10x performance improvement. For Colab or local environments where these bottlenecks don't exist, it maintains standard settings with progress bars and parallel data loading. This automatic adaptation means you can run the same notebook efficiently in both environments without manual configuration changes, ensuring optimal performance regardless of where you execute your training.

In [ ]:
import os
import sys

def is_remote_vscode():
    """Detect if running in VSCode remote container/SSH environment.

    Uses DMI Product information to distinguish between VSCode Remote and Google Colab.
    Google Colab runs on "Google Compute Engine" VMs.
    """
    try:
        if os.path.exists('/sys/class/dmi/id/product_name'):
            with open('/sys/class/dmi/id/product_name', 'r') as f:
                product = f.read().strip()
                print("DMI:", product)
                # Google Colab runs on Google Compute Engine
                if 'Google' in product or 'Compute Engine' in product:
                    return False  # This is Google Colab
        # If not Google Compute Engine, assume VSCode Remote/Local
        return True
    except:
        # If we can't read DMI, assume local/non-Colab
        return True

# Detect environment and set optimal configurations
IS_REMOTE = is_remote_vscode()
print(f"Environment detected: {'VSCode Remote' if IS_REMOTE else 'Google Colab'}")

# Performance-optimized settings based on environment
if IS_REMOTE:
    NUM_WORKERS = 2  #  to avoid IPC overhead
    PIN_MEMORY = False  # Reduce memory pressure in container
    PERSISTENT_WORKERS = False
    PREFETCH_FACTOR = None
    ENABLE_PROGRESS_BAR = False  # Disable progress bar updates over network
    LOG_EVERY_N_STEPS = 50  # Reduce logging frequency
    CALLBACK_VERBOSE = True
else:
    NUM_WORKERS = 4  # Leverage multiple cores for data loading
    PIN_MEMORY = True  # Faster GPU transfer
    PERSISTENT_WORKERS = True  # Reuse worker processes
    PREFETCH_FACTOR = 2  # Load batches ahead
    ENABLE_PROGRESS_BAR = True
    LOG_EVERY_N_STEPS = 10
    CALLBACK_VERBOSE = True
print(f"DataLoader num_workers set to: {NUM_WORKERS}")
print("Progress bar set to", ENABLE_PROGRESS_BAR)

## Data Preparation

This code prepares text data for a character-level RNN by converting names into one-hot encoded tensors:


In [ ]:
# Character set and conversion functions
#Defines the vocabulary: a-z, A-Z, space, punctuation, and "_" for unknown characters.
allowed_characters = string.ascii_letters + " .,;'" + "_"
n_letters = len(allowed_characters)   # 58 characters total

def unicodeToAscii(s):
    """Convert Unicode string to plain ASCII.
    Normalizes Unicode text to ASCII (e.g., "Ślusàrski" → "Slusarski"):
    normalize('NFD'): Decomposes accented characters (é → e + ´)
    Filters out accent marks (category 'Mn')
    Keeps only allowed characters
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in allowed_characters
    )

def letterToIndex(letter):
    """Find letter index from allowed_characters.
    Maps each character to its position in the vocabulary:
    'a' → 0, 'b' → 1, etc.
    Unknown characters → index of "_"
    """
    if letter not in allowed_characters:
        return allowed_characters.find("_")
    else:
        return allowed_characters.find(letter)

def lineToTensor(line):
    """Turn a line into a <line_length x 1 x n_letters> tensor.
    Converts a string into a one-hot encoded tensor of shape [length, 1, 58]:
    Each character becomes a vector with all zeros except a 1 at its index position
    "Ab" → [[1,0,0,...], [0,1,0,...]] (2 timesteps, each with 58-dim vector)
    The middle dimension (1) represents batch size
    """
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(f"Number of characters: {n_letters}")
print(f"Example conversion: 'Ślusàrski' -> '{unicodeToAscii('Ślusàrski')}'")

In [ ]:
class NamesDataset(Dataset):
    """Dataset for name classification."""

    def __init__(self, data_dir):
        self.data_dir = data_dir
        labels_set = set()

        self.data = []
        self.labels = []

        # Read all .txt files in the specified directory
        text_files = glob.glob(os.path.join(data_dir, '*.txt'))
        for filename in sorted(text_files):  # Sort for reproducibility
            label = os.path.splitext(os.path.basename(filename))[0]
            labels_set.add(label)
            lines = open(filename, encoding='utf-8').read().strip().split('\n')
            for name in lines:
                self.data.append(unicodeToAscii(name))
                self.labels.append(label)

        # Create label to index mapping
        self.labels_uniq = sorted(list(labels_set))
        self.label_to_idx = {label: idx for idx, label in enumerate(self.labels_uniq)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        name = self.data[idx]
        label = self.labels[idx]

        # Convert to tensors (one hot encoding)
        #Ana ->one-hot encoding for each char
        name_tensor = lineToTensor(name)
        label_idx = self.label_to_idx[label]

        return name_tensor, label_idx

    @property
    def num_classes(self):
        return len(self.labels_uniq)

## Lightning Data Module

We'll create a LightningDataModule to handle data loading with train/val/test splits.

collate_fn is called by PyTorch's DataLoader to combine multiple samples into a single batch:

1. Gets variable-length sequences: "Ann" (3 chars), "Alexander" (9 chars), "Li" (2 chars)

2. Pads to max length: All sequences padded to length 9:

"Ann" → 3 real chars + 6 padding zeros

"Alexander" → 9 real chars

"Li" → 2 real chars + 7 padding zeros

3. Tracks original lengths: [3, 9, 2] - needed so the RNN knows which values are padding

4. Returns batch-ready tensors:

padded_names: shape [9, 3, 58] (max_len, batch_size, features)

labels_tensor: shape [3]

lengths: [3, 9, 2]

With padding + batch processing, the RNN can process 3 (or 64) names simultaneously instead of one-at-a-time, achieving massive speedup on GPUs while still respecting the true length of each sequence (using pack_padded_sequence in the forward pass).

In [ ]:
def collate_fn(batch):
    """Custom collate function to handle variable length sequences with padding.

    Pads sequences to the same length and returns lengths for pack_padded_sequence.
    """
    names, labels = zip(*batch)

    # Get sequence lengths (before padding)
    lengths = torch.tensor([name.size(0) for name in names], dtype=torch.long)

    # Pad sequences: pad_sequence expects (seq_len, batch, features)
    # Our tensors are (seq_len, 1, 58), so we need to squeeze and then pad
    names_squeezed = [name.squeeze(1) for name in names]  # Remove middle dimension
    padded_names = pad_sequence(names_squeezed, batch_first=False, padding_value=0.0)
    # Result shape: (max_seq_len, batch_size, 58)

    labels_tensor = torch.tensor(labels, dtype=torch.long)

    return padded_names, labels_tensor, lengths


class NamesDataModule(pl.LightningDataModule):
    """Lightning DataModule for names dataset."""

    def __init__(self, data_dir: str, batch_size: int = 32, num_workers: int = 0,
                 train_ratio: float = 0.7, val_ratio: float = 0.15, test_ratio: float = 0.15):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_ratio = train_ratio
        self.val_ratio = val_ratio
        self.test_ratio = test_ratio

    def setup(self, stage=None):
        """Load and split dataset."""
        # Load full dataset
        full_dataset = NamesDataset(self.data_dir)
        self.labels_uniq = full_dataset.labels_uniq
        self.num_classes = full_dataset.num_classes

        # Calculate split sizes
        total_size = len(full_dataset)
        train_size = int(self.train_ratio * total_size)
        val_size = int(self.val_ratio * total_size)
        test_size = total_size - train_size - val_size

        # Split dataset
        self.train_dataset, self.val_dataset, self.test_dataset = random_split(
            full_dataset,
            [train_size, val_size, test_size],
            generator=torch.Generator().manual_seed(2024)
        )

        print(f"Dataset split: Train={len(self.train_dataset)}, "
              f"Val={len(self.val_dataset)}, Test={len(self.test_dataset)}")
        print(f"Number of classes: {self.num_classes}")

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=collate_fn,
            num_workers=self.num_workers,
            pin_memory=PIN_MEMORY,
            persistent_workers=True if self.num_workers > 0 else False
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=collate_fn,
            num_workers=self.num_workers,
            pin_memory=PIN_MEMORY,
            persistent_workers=True if self.num_workers > 0 else False
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=collate_fn,
            num_workers=self.num_workers,
            pin_memory=PIN_MEMORY,
            persistent_workers=True if self.num_workers > 0 else False
        )

## Lightning Module

We'll implement the CharRNN model as a LightningModule with training, validation, and test steps.


### Understanding RNN Outputs: Why We Use `hidden[0]` Instead of Last Output

When working with RNNs for sequence classification, a common question arises: **Why do we use the hidden state (`hidden[0]`) instead of extracting the last output from the RNN?**

This is an important concept to understand, especially when dealing with variable-length sequences and padding. Let's break it down step by step.

#### What Does an RNN Return?

When you call an RNN in PyTorch, it returns two things:

```python
packed_output, hidden = self.rnn(packed_input)
```

1. **`packed_output`**: A `PackedSequence` containing the RNN's output at **every timestep**
   - When unpacked, shape: `(seq_len, batch_size, hidden_size)`
   - Contains the output computed at each position in the sequence
   - For packed sequences, only contains values for non-padded positions

2. **`hidden`**: The **final hidden state** tensor
   - Shape: `(num_layers, batch_size, hidden_size)`
   - For our single-layer RNN: `(1, batch_size, hidden_size)`
   - Contains the hidden state after processing the entire sequence

#### The Challenge with Variable-Length Sequences

Consider a batch with three names of different lengths:
- **Sequence 1**: "Bob" (length 3)
- **Sequence 2**: "Alexandra" (length 9)
- **Sequence 3**: "Li" (length 2)

After padding all sequences to the maximum length (9 characters), our padded batch looks like this:

```
Position:  0  1  2  3  4  5  6  7  8
─────────────────────────────────────
Seq 1:     B  o  b  [P][P][P][P][P][P]    [P] = padding
Seq 2:     A  l  e  x  a  n  d  r  a
Seq 3:     L  i  [P][P][P][P][P][P][P]
```

Now, here's the critical question: **Where is the "last output" for each sequence?**

- For "Bob": The last **real** output is at position 2 (after "b")
- For "Alexandra": The last real output is at position 8 (after "a")
- For "Li": The last real output is at position 1 (after "i")

The "last output" is at **different positions** for different sequences!

#### Why We Can't Simply Use `packed_output[-1]`

If we naively tried to extract the last output like this:

```python
# WRONG APPROACH!
last_output = packed_output[-1]  # Gets output at position 8 for ALL sequences
```

This would give us:
- For "Bob": Output at position 8 → **padding garbage!** ❌
- For "Alexandra": Output at position 8 → correct final output ✓
- For "Li": Output at position 8 → **padding garbage!** ❌

This doesn't work! We'd need to manually index each sequence by its actual length:

```python
# Inefficient and error-prone approach
for i in range(batch_size):
    last_output_i = packed_output[lengths[i]-1, i, :]  # Manual indexing by length
```

This defeats the purpose of batching and is cumbersome.

#### The Elegant Solution: Using `hidden[0]`

PyTorch's RNN has a clever solution: the `hidden` tensor **automatically contains the final hidden state** for each sequence, correctly handling variable lengths when you use `pack_padded_sequence`.

```python
hidden[0]  # Shape: (batch_size, hidden_size)
```

What does `hidden[0]` contain?
- `hidden[0][0]` → Final hidden state after processing "b" (position 2 for Seq 1) ✓
- `hidden[0][1]` → Final hidden state after processing "a" (position 8 for Seq 2) ✓
- `hidden[0][2]` → Final hidden state after processing "i" (position 1 for Seq 3) ✓

**Perfect!** Each row contains the correct final state, regardless of padding.

#### But Wait—Aren't They the Same Thing?

You might wonder: "Isn't the last output the same as the final hidden state?"

**Conceptually, yes!** For a single-layer RNN, the output at timestep `t` is derived from the hidden state at timestep `t`. At the last real timestep of a sequence, they represent the same information.

**Practically, no!** The key differences are:

1. **Location**:
   - Last outputs are scattered at different indices in `packed_output`
   - Final hidden states are neatly collected in `hidden[0]`

2. **Convenience**:
   - `hidden[0]` gives you shape `(batch_size, hidden_size)` directly
   - Extracting from `packed_output` requires manual indexing by sequence lengths

3. **Efficiency**:
   - Using `hidden[0]` is a single tensor slice
   - Manual extraction requires a loop or complex indexing

4. **Best Practice**:
   - `hidden[0]` is the idiomatic PyTorch pattern for sequence classification
   - It's what you'll see in tutorials, papers, and production code

#### Summary

When we write:
```python
output = self.h2o(hidden[0])
```

We're using `hidden[0]` because:
- ✅ It automatically contains the final state after each sequence's last real character
- ✅ It handles variable-length sequences correctly with `pack_padded_sequence`
- ✅ It's efficient and doesn't require manual indexing
- ✅ It's the standard PyTorch pattern for sequence classification

This is exactly what we want for classifying names—the network's final representation of each sequence, ready to be transformed into class predictions!

In [ ]:
class CharRNNLightning(pl.LightningModule):
    """Character-level RNN for name classification using PyTorch Lightning."""

    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.005):
        super().__init__()
        self.save_hyperparameters()

        self.rnn = nn.RNN(input_size, hidden_size, batch_first=False)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.learning_rate = learning_rate

        # Loss function
        self.criterion = nn.CrossEntropyLoss()

        # Metrics
        self.train_acc = Accuracy(task="multiclass", num_classes=output_size)
        self.val_acc = Accuracy(task="multiclass", num_classes=output_size)
        self.test_acc = Accuracy(task="multiclass", num_classes=output_size)

        self.train_f1 = F1Score(task="multiclass", num_classes=output_size, average='weighted')
        self.val_f1 = F1Score(task="multiclass", num_classes=output_size, average='weighted')
        self.test_f1 = F1Score(task="multiclass", num_classes=output_size, average='weighted')

    def forward(self, x, lengths):
        """Forward pass through the network.

        Args:
            x: Padded input tensor of shape (seq_len, batch_size, input_size)
            lengths: Tensor of sequence lengths for pack_padded_sequence

        Returns:
            Output tensor of shape (batch_size, output_size)
        """
        # Pack padded sequences for efficient RNN processing
        packed_input = pack_padded_sequence(x, lengths.cpu(), enforce_sorted=False)

        # Run through RNN
        packed_output, hidden = self.rnn(packed_input)

        # hidden shape: (num_layers, batch_size, hidden_size)
        # We use hidden[0] which is shape (batch_size, hidden_size)
        output = self.h2o(hidden[0])

        return output

    def training_step(self, batch, batch_idx):
        """Training step."""
        names, labels, lengths = batch

        # Single forward pass for entire batch
        output = self(names, lengths)
        loss = self.criterion(output, labels)

        # Compute metrics using torchmetrics
        preds = output.argmax(dim=1)
        self.train_acc(preds, labels)
        self.train_f1(preds, labels)

        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, batch_size=len(labels))
        self.log('train_acc', self.train_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('train_f1', self.train_f1, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """Validation step."""
        names, labels, lengths = batch

        # Single forward pass for entire batch
        output = self(names, lengths)
        loss = self.criterion(output, labels)

        # Compute metrics using torchmetrics
        preds = output.argmax(dim=1)
        self.val_acc(preds, labels)
        self.val_f1(preds, labels)

        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, batch_size=len(labels))
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val_f1', self.val_f1, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        """Test step."""
        names, labels, lengths = batch

        # Single forward pass for entire batch
        output = self(names, lengths)
        loss = self.criterion(output, labels)

        # Compute metrics using torchmetrics
        preds = output.argmax(dim=1)
        self.test_acc(preds, labels)
        self.test_f1(preds, labels)

        self.log('test_loss', loss, on_step=False, on_epoch=True, batch_size=len(labels))
        self.log('test_acc', self.test_acc, on_step=False, on_epoch=True)
        self.log('test_f1', self.test_f1, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        """Configure optimizer."""
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)

        # Learning rate scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=5
        )

        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',
                'interval': 'epoch',
                'frequency': 1
            }
        }

In [ ]:
import time

class SimpleEpochProgressCallback(pl.Callback):
    """Lightweight callback to show epoch progress without progress bar overhead."""

    def __init__(self):
        super().__init__()
        self.epoch_start_time = None

    def on_train_start(self, trainer, pl_module):
        """Print training configuration at the start."""
        train_dataloader = trainer.train_dataloader

        # Get batch size and number of batches
        if hasattr(train_dataloader, 'batch_size'):
            batch_size = train_dataloader.batch_size
        elif hasattr(train_dataloader, 'loaders') and hasattr(train_dataloader.loaders, 'batch_size'):
            batch_size = train_dataloader.loaders.batch_size
        else:
            batch_size = getattr(train_dataloader, 'batch_size', 'N/A')

        num_batches = len(train_dataloader)

        print(f"\nTraining Configuration: Batch Size={batch_size}, Steps per Epoch={num_batches}\n")

    def on_train_epoch_start(self, trainer, pl_module):
        """Record the start time of the epoch."""
        self.epoch_start_time = time.time()

    def on_train_epoch_end(self, trainer, pl_module):
        """Print single-line summary at the end of each training epoch."""
        current_epoch = trainer.current_epoch + 1
        max_epochs = trainer.max_epochs

        # Calculate epoch duration
        epoch_time = time.time() - self.epoch_start_time if self.epoch_start_time else 0

        # Get metrics from logged values
        train_loss = trainer.callback_metrics.get('train_loss', None)
        val_loss = trainer.callback_metrics.get('val_loss', None)
        val_acc = trainer.callback_metrics.get('val_acc', None)

        # Build single-line output
        parts = [f"Epoch {current_epoch}/{max_epochs}"]

        if train_loss is not None:
            parts.append(f"Train Loss: {train_loss:.4f}")
        if val_loss is not None:
            parts.append(f"Val Loss: {val_loss:.4f}")
        if val_acc is not None:
            parts.append(f"Val Acc: {val_acc:.4f}")

        parts.append(f"Time: {epoch_time:.2f}s")

        print(" | ".join(parts))

## Training

Now we'll set up the trainer with callbacks and train the model.

In [ ]:
# Initialize data module
data_module = NamesDataModule(
    data_dir='data/names',
    batch_size=32,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    num_workers=NUM_WORKERS
)

# Setup data
data_module.setup()

In [ ]:
# Initialize model
model = CharRNNLightning(
    input_size=n_letters,
    hidden_size=128,
    output_size=data_module.num_classes,
    learning_rate=0.005
)

print(model)

## 🔢 Understanding RNN Parameter Calculation

When you see `RNN(58, 128)` in the model architecture above, you might wonder: **How many parameters does this RNN layer actually have?**

### The Four Components of RNN Parameters

A vanilla RNN layer contains **four sets of learnable parameters**:

1. **Input-to-Hidden Weights (W_ih)**: `input_size × hidden_size`
   - Transforms the input at each timestep to hidden state dimension
   - For our model: `58 × 128 = 7,424` parameters

2. **Hidden-to-Hidden Weights (W_hh)**: `hidden_size × hidden_size`
   - Transforms the previous hidden state to the current hidden state
   - For our model: `128 × 128 = 16,384` parameters

3. **Input-to-Hidden Bias (b_ih)**: `hidden_size`
   - Bias term for the input transformation
   - For our model: `128` parameters

4. **Hidden-to-Hidden Bias (b_hh)**: `hidden_size`
   - Bias term for the hidden state transformation
   - For our model: `128` parameters

### The Formula

**Total RNN parameters = (input_size × hidden_size) + (hidden_size × hidden_size) + (2 × hidden_size)**

Or more compactly:

**Total RNN parameters = hidden_size × (input_size + hidden_size + 2)**

### Calculation for This Model

```
RNN layer parameters = 128 × (58 + 128 + 2)
                     = 128 × 188
                     = 24,064 parameters
```

### Complete Model Parameters

Our complete model has two components:

| Component | Calculation | Parameters |
|-----------|-------------|------------|
| **RNN layer** | `128 × (58 + 128 + 2)` | **24,064** |
| **Linear layer (h2o)** | `(128 × 18) + 18` | **2,322** |
| **Total** | | **26,386** |

The linear layer has:
- Weight matrix: `128 × 18 = 2,304` parameters
- Bias vector: `18` parameters

### The Math Behind It

The RNN computes the hidden state at each timestep using this formula:

```
h_t = tanh(W_ih @ x_t + b_ih + W_hh @ h_{t-1} + b_hh)
```

Where:
- `x_t` is the input at timestep t (shape: `[58]`)
- `h_{t-1}` is the previous hidden state (shape: `[128]`)
- `h_t` is the current hidden state (shape: `[128]`)
- `W_ih` is `[128, 58]` → 7,424 parameters
- `W_hh` is `[128, 128]` → 16,384 parameters
- `b_ih` is `[128]` → 128 parameters
- `b_hh` is `[128]` → 128 parameters

This gives us **24,064 parameters for just the RNN layer**.

### Why This Matters

Understanding parameter counts helps you:
1. **Compare architectures**: LSTM has 4× more parameters than RNN for the same hidden size
2. **Estimate memory**: Each parameter typically requires 4 bytes (float32)
3. **Prevent overfitting**: More parameters need more training data
4. **Debug models**: Unexpected parameter counts can reveal architecture mistakes

For example, in this notebook:
- **Vanilla RNN**: 24,064 parameters (efficient but simple)
- **LSTM**: ~74K parameters (4× more, better at long sequences)
- **GRU**: ~56K parameters (3× more, good balance)

The RNN's simplicity (fewer parameters) makes it fast to train but limits its ability to capture complex temporal dependencies.

### Quick Verification

You can verify this calculation with PyTorch:

In [ ]:
# Count only RNN parameters
rnn_params = sum(p.numel() for p in model.rnn.parameters())
print(f"RNN parameters: {rnn_params:,}")  # Output: 24,064

# Count only Linear layer parameters
linear_params = sum(p.numel() for p in model.h2o.parameters())
print(f"Linear parameters: {linear_params:,}")  # Output: 2,322

# Total model parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")  # Output: 26,386

In [ ]:
# Configure callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    filename='char-rnn-{epoch:02d}-{val_loss:.2f}',
    monitor='val_loss',
    mode='min',
    save_top_k=3,
    verbose=CALLBACK_VERBOSE
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min',
    verbose=CALLBACK_VERBOSE
)

# CSV logger for easy plotting
csv_logger = CSVLogger('logs/', name='char_rnn')

In [ ]:
# Add epoch progress callback to callbacks list
epoch_progress = SimpleEpochProgressCallback()

# Initialize trainer
trainer = pl.Trainer(
    max_epochs=50,
    callbacks=[checkpoint_callback, early_stopping, epoch_progress],
    logger=csv_logger,
    deterministic=True,
    log_every_n_steps=LOG_EVERY_N_STEPS,
    enable_progress_bar=ENABLE_PROGRESS_BAR
)

# Train the model
trainer.fit(model, data_module)

## Plotting Training Curves

Let's visualize the training and validation loss over epochs.

In [ ]:
# Load metrics from CSV logger
metrics = pd.read_csv(f'{csv_logger.log_dir}/metrics.csv')

# Aggregate by epoch (remove NaN values)
train_loss = metrics[['epoch', 'train_loss']].dropna()
val_loss = metrics[['epoch', 'val_loss']].dropna()
val_acc = metrics[['epoch', 'val_acc']].dropna()

# Create figure with subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
ax1.plot(train_loss['epoch'], train_loss['train_loss'], label='Train Loss', marker='o')
ax1.plot(val_loss['epoch'], val_loss['val_loss'], label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot accuracy
ax2.plot(val_acc['epoch'], val_acc['val_acc'], label='Val Accuracy', marker='s', color='green')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Training Loss: {train_loss['train_loss'].iloc[-1]:.4f}")
print(f"Final Validation Loss: {val_loss['val_loss'].iloc[-1]:.4f}")
print(f"Final Validation Accuracy: {val_acc['val_acc'].iloc[-1]:.4f}")

## Test Evaluation

Finally, let's evaluate the model on the test set.

In [ ]:
# Test the model
test_results = trainer.test(model, data_module, ckpt_path='best')

print(f"\nTest Results:")
print(f"Test Loss: {test_results[0]['test_loss']:.4f}")
print(f"Test Accuracy: {test_results[0]['test_acc']:.4f}")
print(f"Test F1 Score: {test_results[0]['test_f1']:.4f}")

## Confusion Matrix

Let's visualize the confusion matrix on the test set to see which languages the model confuses with each other.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.ticker as ticker

def compute_confusion_matrix(model, dataloader, labels):
    """Compute confusion matrix for the given dataloader."""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            names, labels_batch, lengths = batch

            # Process entire batch at once
            output = model(names, lengths)
            preds = output.argmax(dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels_batch.cpu().numpy())

    return confusion_matrix(all_labels, all_preds), all_preds, all_labels

# Compute confusion matrix on test set
conf_matrix, predictions, true_labels = compute_confusion_matrix(
    model,
    data_module.test_dataloader(),
    data_module.labels_uniq
)

# Normalize by row (true labels)
conf_matrix_normalized = conf_matrix.astype('float')
for i in range(len(data_module.labels_uniq)):
    row_sum = conf_matrix_normalized[i].sum()
    if row_sum > 0:
        conf_matrix_normalized[i] = conf_matrix_normalized[i] / row_sum

# Plot confusion matrix
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Plot raw counts
im1 = ax1.matshow(conf_matrix, cmap='Blues')
ax1.set_title('Confusion Matrix (Counts)', pad=20, fontsize=14, fontweight='bold')
ax1.set_xlabel('Predicted Label', fontsize=12)
ax1.set_ylabel('True Label', fontsize=12)
ax1.set_xticks(np.arange(len(data_module.labels_uniq)))
ax1.set_yticks(np.arange(len(data_module.labels_uniq)))
ax1.set_xticklabels(data_module.labels_uniq, rotation=90)
ax1.set_yticklabels(data_module.labels_uniq)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax1.yaxis.set_major_locator(ticker.MultipleLocator(1))
plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)

# Plot normalized (percentages)
im2 = ax2.matshow(conf_matrix_normalized, cmap='Blues', vmin=0, vmax=1)
ax2.set_title('Confusion Matrix (Normalized)', pad=20, fontsize=14, fontweight='bold')
ax2.set_xlabel('Predicted Label', fontsize=12)
ax2.set_ylabel('True Label', fontsize=12)
ax2.set_xticks(np.arange(len(data_module.labels_uniq)))
ax2.set_yticks(np.arange(len(data_module.labels_uniq)))
ax2.set_xticklabels(data_module.labels_uniq, rotation=90)
ax2.set_yticklabels(data_module.labels_uniq)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax2.yaxis.set_major_locator(ticker.MultipleLocator(1))
plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

# Calculate and display per-class accuracy
print("\nPer-class Accuracy on Test Set:")
print("-" * 50)
for i, label in enumerate(data_module.labels_uniq):
    accuracy = conf_matrix_normalized[i, i]
    print(f"{label:15s}: {accuracy:.2%} ({conf_matrix[i, i]}/{conf_matrix[i].sum()})")

# Find most common confusions
print("\nMost Common Confusions (excluding correct predictions):")
print("-" * 50)
confusions = []
for i in range(len(data_module.labels_uniq)):
    for j in range(len(data_module.labels_uniq)):
        if i != j and conf_matrix[i, j] > 0:
            confusions.append((
                data_module.labels_uniq[i],
                data_module.labels_uniq[j],
                conf_matrix[i, j],
                conf_matrix_normalized[i, j]
            ))

# Sort by count and show top 10
confusions.sort(key=lambda x: x[2], reverse=True)
for true_label, pred_label, count, percentage in confusions[:10]:
    print(f"{true_label:15s} → {pred_label:15s}: {count:4d} times ({percentage:.1%})")

## Inference Example

Let's test the model with some example names.

In [ ]:
def predict_name(model, name, labels):
    """Predict the language of origin for a given name."""
    model.eval()
    with torch.no_grad():
        name_tensor = lineToTensor(name)#.squeeze(1)  # Remove batch dim: (seq_len, 58)
       # name_tensor = name_tensor.unsqueeze(1)  # Add batch dim: (seq_len, 1, 58)
        lengths = torch.tensor([name_tensor.size(0)], dtype=torch.long)

        output = model(name_tensor, lengths)
        probabilities = F.softmax(output, dim=1)
        top_prob, top_idx = probabilities.topk(3)

        print(f"\nName: {name}")
        print("Top 3 predictions:")
        for i in range(3):
            label = labels[top_idx[0][i].item()]
            prob = top_prob[0][i].item()
            print(f"  {i+1}. {label}: {prob:.2%}")

# Test with example names
test_names = ['Rodriguez', 'Smith', 'Wang', 'O\'Brien', 'Kowalski', 'Nakamura']

for name in test_names:
    predict_name(model, name, data_module.labels_uniq)

## Summary

This notebook demonstrates:
1. **PyTorch Lightning best practices**: Using LightningModule and LightningDataModule for clean, organized code
2. **Proper data splitting**: 70% train, 15% validation, 15% test
3. **Training monitoring**: Logging train and validation metrics, with plots
4. **Callbacks**: Model checkpointing and early stopping
5. **Reproducibility**: Fixed random seeds throughout
6. **Learning rate scheduling**: ReduceLROnPlateau for adaptive learning


## 🎯 Activity 1: Implement LSTM for Name Classification

### Objective
Modify the character-level RNN model to use **LSTM (Long Short-Term Memory)** architecture and compare its performance with the vanilla RNN.

### Background
LSTM is an advanced recurrent architecture that addresses the vanishing gradient problem in vanilla RNNs. It uses three gates to control information flow:
- **Input gate**: Controls what new information to add to the cell state
- **Forget gate**: Controls what information to discard from the cell state
- **Output gate**: Controls what information to output from the cell state

This gating mechanism allows LSTMs to capture long-term dependencies more effectively than vanilla RNNs.

### Your Task
Create a new model class `CharLSTMLightning` that uses `nn.LSTM` instead of `nn.RNN` to classify names by language of origin.

### Key Differences to Handle
1. **Module replacement**: Replace `nn.RNN` with `nn.LSTM` in your model definition
2. **Output format**: LSTM returns `(output, (hidden, cell))` while RNN returns `(output, hidden)`
   - You'll need to unpack the tuple: `packed_output, (hidden, cell) = self.lstm(packed_input)`
3. **Final state extraction**: After unpacking, use `hidden[0]` to get the final hidden state for classification

### Hints
- Start by copying the `CharRNNLightning` class and renaming it to `CharLSTMLightning`
- Change `self.rnn` to `self.lstm` and use `nn.LSTM` instead of `nn.LSTM`
- In the forward method, handle the LSTM's tuple output: `(hidden, cell)`
- Everything else remains the same!

### Expected Outcome
After training, you should achieve similar or better test accuracy compared to the vanilla RNN (~79-82%). LSTMs often perform better on sequence tasks due to their ability to maintain long-term memory.

In [ ]:
# Your code here

## 🎯 Activity 2: Implement GRU for Name Classification

### Objective
Modify the character-level RNN model to use **GRU (Gated Recurrent Unit)** architecture and compare its performance with both vanilla RNN and LSTM.

### Background
GRU is a recurrent architecture introduced as a simpler alternative to LSTM. It uses only two gates instead of three:
- **Reset gate**: Controls how much of the previous hidden state to forget
- **Update gate**: Controls how much of the new content to add to the hidden state

GRU combines the forget and input gates of LSTM into a single update gate, making it computationally more efficient while maintaining similar performance. GRUs have fewer parameters than LSTMs, which can lead to faster training and less overfitting on smaller datasets.

### Your Task
Create a new model class `CharGRULightning` that uses `nn.GRU` instead of `nn.RNN` to classify names by language of origin.

### Key Differences to Handle
1. **Module replacement**: Replace `nn.RNN` with `nn.GRU` in your model definition
2. **Output format**: GRU returns `(output, hidden)` just like vanilla RNN (no cell state!)
   - The interface is identical: `packed_output, hidden = self.gru(packed_input)`
3. **Final state extraction**: Use `hidden[0]` to get the final hidden state (same as RNN)

### Hints
- Start by copying the `CharRNNLightning` class and renaming it to `CharGRULightning`
- Change `self.rnn` to `self.gru` and use `nn.GRU` instead of `nn.RNN`
- The forward method is almost identical to the RNN version (no tuple unpacking needed like LSTM!)
- GRU is a drop-in replacement for RNN with better performance

### Expected Outcome
After training, you should achieve similar or better test accuracy compared to both vanilla RNN and LSTM (~79-82%). GRUs often match LSTM performance while being more efficient to train.

In [ ]:
# Your code here